### Comparison between the A/E method and the Deep Learning autoencoder + classifier method
This notebooks allows for comparing the two SSE/MSE classifiers in terms of S/B gain for peaks at different, customizable energies. For each peak, the Comparison class carries out the following analysis steps:
- Selecting an energy region centered at the peak value +- a certain energy span.
- Selecting MSE events inside the energy region with a first method ("nn" or "avse"), looping through a set of thresholds. The threshold for the second method is adjusted in order to match the same event number as the first method.
- Estimating the S/B ratio and its standard deviation with robust fitting for each peak and threshold value.

In [1]:
from begepro.autoencoder.sn_analysis import Comparison
from begepro.autoencoder.dataloading import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt

In [2]:
opts = {"loadpath": "/mnt/e/BEGE_data/waveforms_crioconite", # where .bin files are stored
"savepath": "../../dataset/Th232", # where .npy files will be saved with readfiles() and loaded with load()
"subsampling_factor": 3}

dloader = DataLoader(opts)
_, _, energies, _, _ = dloader.load()

opts_2 = {"loadpath": "/mnt/e/BEGE_data/waveforms_crioconite", # where .bin files are stored
"savepath": "../../dataset/Th232_2", # where .npy files will be saved with readfiles() and loaded with load()
"subsampling_factor": 3}

dloader_2 = DataLoader(opts_2)
_, _, energies_2, _, _ = dloader_2.load()

In [3]:
preds = np.load("../scripts/preds.npy")
ratio = np.load("../scripts/ratio.npy")
preds_2 = np.load("../scripts/preds_2.npy")
ratio_2 = np.load("../scripts/ratio_2.npy")

energies = np.concatenate([energies, energies_2])
preds = np.concatenate([preds, preds_2])
ratio = np.concatenate([ratio, ratio_2])

In [4]:
comparison = Comparison(energies, ratio, preds)
M_ELECTRON = 511
cs = np.linspace(0.015, 0.025, 30)
peaks = [2615, 2615 - 1*M_ELECTRON, 2615 - 2*M_ELECTRON, 1588, 1622, 1460, 860, 835, 409, 583, 2177]


In [5]:
sb_nn, std_sb_nn, sb_avse, std_sb_avse = comparison.compare(peaks, cs, benchmark = "avse")

Estimating S/B for peak at 2615 keV
Estimating S/B for peak at 2104 keV
Estimating S/B for peak at 1593 keV
Estimating S/B for peak at 1588 keV
Estimating S/B for peak at 1622 keV
Estimating S/B for peak at 1460 keV
Estimating S/B for peak at 860 keV
Estimating S/B for peak at 835 keV
Estimating S/B for peak at 409 keV
Estimating S/B for peak at 583 keV
Estimating S/B for peak at 2177 keV


In [7]:
for n in range(len(peaks)):
    plt.plot(cs, sb_nn[:,n])
    plt.fill_between(cs, sb_nn[:,n] + std_sb_nn[:,n], sb_nn[:,n] - std_sb_nn[:,n],label='_nolegend_', alpha = 0.5)
    plt.plot(cs, sb_avse[:,n])
    plt.fill_between(cs, sb_avse[:,n] + std_sb_avse[:,n], sb_avse[:,n] - std_sb_avse[:,n],label='_nolegend_', alpha = 0.5)
    plt.xlabel("NN score threshold")
    plt.ylabel("S/B ratio")
    plt.title(f"Peak at {peaks[n]}keV")
    plt.legend(["NN", "A/E"])
    plt.savefig(f"../../results/z_peak_at_{peaks[n]}_122_sampling.svg")
    plt.clf()

<Figure size 640x480 with 0 Axes>